In [2]:
import numpy as np

def apply_pbc(coords, lattice_vectors):
    """
    Apply periodic boundary conditions to bring coordinates inside the unit cell.

    Parameters:
    coords (numpy.ndarray): The coordinates of the atoms, shape (N, 3).
    lattice_vectors (numpy.ndarray): The lattice vectors, shape (3, 3).

    Returns:
    numpy.ndarray: The coordinates wrapped inside the unit cell.
    """
    # Convert coordinates to fractional coordinates
    fractional_coords = np.linalg.solve(lattice_vectors.T, coords.T).T

    # Apply periodic boundary conditions: bring fractional coordinates into the range [0, 1)
    fractional_coords -= np.floor(fractional_coords)

    # Convert back to cartesian coordinates
    wrapped_coords = fractional_coords.dot(lattice_vectors)

    return wrapped_coords

# Example usage
coords = np.array([
    [3.5, 4.5, 5.5],  # Example coordinates that might be outside the unit cell
    [-0.5, 1.5, 2.5],
    [6.0, 6.0, 6.0]
])

lattice_vectors = np.array([
    [1.0, 0.0, 0.0],  # Lattice vector a
    [0.0, 1.0, 0.0],  # Lattice vector b
    [0.0, 0.0, 1.0]   # Lattice vector c
])

wrapped_coords = apply_pbc(coords, lattice_vectors)
print("Original coordinates:")
print(coords)
print("Wrapped coordinates:")
print(wrapped_coords)

Original coordinates:
[[ 3.5  4.5  5.5]
 [-0.5  1.5  2.5]
 [ 6.   6.   6. ]]
Wrapped coordinates:
[[0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.  0.  0. ]]


In [4]:
print("test")

test


In [5]:
import torch
def apply_pbc_torch(coords, lattice_vectors):
    """
    Apply periodic boundary conditions to bring coordinates inside the unit cell.

    Parameters:
    coords (torch.Tensor): The coordinates of the atoms, shape (N, 3).
    lattice_vectors (torch.Tensor): The lattice vectors, shape (3, 3).

    Returns:
    torch.Tensor: The coordinates wrapped inside the unit cell.
    """
    # Convert coordinates to fractional coordinates
    inv_lattice_vectors = torch.linalg.inv(lattice_vectors)
    fractional_coords = torch.matmul(coords, inv_lattice_vectors.T)

    # Apply periodic boundary conditions: bring fractional coordinates into the range [0, 1)
    fractional_coords = fractional_coords - torch.floor(fractional_coords)

    # Convert back to cartesian coordinates
    wrapped_coords = torch.matmul(fractional_coords, lattice_vectors)

    return wrapped_coords

# Example usage
coords = torch.tensor([
    [3.5, 4.5, 5.5],  # Example coordinates that might be outside the unit cell
    [-0.5, 1.5, 2.5],
    [6.0, 6.0, 6.0]
])

lattice_vectors = torch.tensor([
    [2.0, 0.0, 0.0],  # Lattice vector a
    [0.0, 2.0, 0.0],  # Lattice vector b
    [0.0, 0.0, 2.0]   # Lattice vector c
])

wrapped_coords = apply_pbc_torch(coords, lattice_vectors)
print("Original coordinates:")
print(coords)
print("Wrapped coordinates:")
print(wrapped_coords)

Original coordinates:
tensor([[ 3.5000,  4.5000,  5.5000],
        [-0.5000,  1.5000,  2.5000],
        [ 6.0000,  6.0000,  6.0000]])
Wrapped coordinates:
tensor([[1.5000, 0.5000, 1.5000],
        [1.5000, 1.5000, 0.5000],
        [0.0000, 0.0000, 0.0000]])


In [41]:
import importlib
import cpmd.asign_wcs_torch
importlib.reload(cpmd.asign_wcs_torch)

# Example usage
atom1 = torch.tensor([1.8, 2.5, 3.5])
atom2 = torch.tensor([4.5, 5.5, 6.5])
lattice_vectors = torch.tensor([
    [2.0, 0.0, 0.0],
    [0.0, 2.0, 0.0],
    [0.0, 0.0, 2.0]
])

wrapped_coords = cpmd.asign_wcs_torch.apply_pbc_torch(torch.stack([atom1, atom2]), lattice_vectors)
relative_vec = cpmd.asign_wcs_torch.relative_vector_torch(wrapped_coords[0], wrapped_coords[1], lattice_vectors)
relative_vec2= cpmd.asign_wcs_torch.relative_vectors_torch2(wrapped_coords[0], wrapped_coords[1], lattice_vectors)

print("Wrapped coordinates:")
print(wrapped_coords)
print("Relative vector considering PBC and minimum image convention:")
print(relative_vec)
print("Relative vector considering PBC and minimum image convention:")
print(relative_vec2)

AttributeError: module 'cpmd.asign_wcs_torch' has no attribute 'relative_vector_torch2'

In [19]:
import ase.io
# メタノールの構造（32分子）を読み込む
data = ase.io.read("../IONS+CENTERS_cell_test.xyz",index=":")

In [20]:
import cpmd.asign_wcs_torch

In [38]:
import importlib

importlib.reload(cpmd.asign_wcs_torch)
print(data[0].get_cell())
position = torch.from_numpy(data[0].get_positions().astype(np.float32))
cell     = torch.from_numpy(data[0].get_cell().astype(np.float32))

pbc_position = cpmd.asign_wcs_torch.apply_pbc_torch(position, cell)
# pbcがかかったか確認
print(torch.any(pbc_position)>12.91197685547878)


Cell([12.91197685547878, 12.91197685547878, 12.91197685547878])
tensor(False)


In [22]:
import ml.atomtype
itp_data=ml.atomtype.read_mol("methanol.mol")
# bonds_list=itp_data.bonds_list
# TODO :: ここで変数を定義してるのはあまりよろしくない．
NUM_MOL_ATOMS=itp_data.num_atoms_per_mol

 -----  ml.read_mol :: parse results... -------
 bonds_list ::  [[0, 4], [0, 2], [0, 3], [1, 0], [5, 1]]
 counter    ::  6
 atom_list  ::  ['C', 'O', 'H', 'H', 'H', 'H']
 -----------------------------------------------
 CH bonds...       [[0, 4], [0, 2], [0, 3]]
 CO bonds...       [[1, 0]]
 OH bonds...       [[5, 1]]
 OO bonds...       []
 CC bonds...       []
 CC ring bonds...  []
 

 ring_bond_index  []
 ch_bond_index    [0, 1, 2]
 oh_bond_index    [4]
 co_bond_index    [3]
 cc_bond_index    []
 O atoms (lonepair)...       [1]
 N atoms (lonepair)...       []
 C atoms ...                 [0]
 H atoms ...                 [2, 3, 4, 5]
  Atomic coordinates 
C 0.94 0.02 -0.09
O 0.47 0.27 -1.4
 -----  ml.read_mol :: parse results... -------
 representative_atom_index  :: 1
 -----------------------------------------------
 coh_index/coc_index :: [oの番号, {coボンドの番号(co_bond_indexの0から数えていくつか),ohボンドの番号}]
 TODO :: もしかしたらbond_indexを使った方が全体的にやりやすいかもしれない
 coh_index :: [[0, {'CO': 0, 'OH': 0}]]
 coc_i

In [23]:
import cpmd.class_atoms_wan
atoms_wan_list = []
for atoms in data: # TODO 最初のatomsのみ利用
    atoms_wan_list.append(cpmd.class_atoms_wan.atoms_wan(atoms,NUM_MOL_ATOMS,itp_data))

In [25]:
atoms_wan_list[0]

In [26]:
for atoms_wan_fr in atoms_wan_list:
    y = lambda x:x._calc_wcs()
    y(atoms_wan_fr)

In [27]:
print(atoms_wan_list[0].)

In [40]:
import importlib

importlib.reload(cpmd.asign_wcs_torch)
#
wfc_list    = torch.from_numpy(np.array(atoms_wan_list[0].wannier).astype(np.float32))
atom_coord  = torch.from_numpy(atoms_wan_list[0].atoms_nowan.get_positions().astype(np.float32))
UNITCELL_VECTORS = torch.from_numpy(atoms_wan_list[0].atoms_nowan.get_cell().astype(np.float32))

# 先頭原子(atom_coord)とWCsの距離ベクトル（PBC，mirror imageを考慮）
wfc_vectors = cpmd.asign_wcs_torch.relative_vector_torch2(atom_coord, wfc_list, UNITCELL_VECTORS)
wfc_distances=torch.linalg.norm(wfc_vectors,axis=1)

wcs_indices = torch.argsort(wfc_distances).reshape(-1)[:1] # 最も近いWCsのインデックスを一つ取り出す．

RuntimeError: The size of tensor a (224) must match the size of tensor b (192) at non-singleton dimension 0